## Special events

In [1]:
from __future__ import print_function

`Button`不用于表示数据类型。相反，`Button`小部件用于处理鼠标点击。`Button`的`on_click`方法可用于注册单击按钮时要调用的函数。`on_click`的文档如下。

In [2]:
import ipywidgets as widgets
print(widgets.Button.on_click.__doc__)

Register a callback to execute when the button is clicked.

        The callback will be called with one argument, the clicked button
        widget instance.

        Parameters
        ----------
        remove: bool (optional)
            Set to true to remove the callback from the list of callbacks.
        


### Example

由于按钮点击是无状态的，它们使用自定义消息从前端传输到后端。 通过使用`on_click`方法，下面显示了一个单击按钮时会打印消息的按钮。

In [3]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Button clicked.
Button clicked.
Button clicked.


## Traitlet事件

Widget属性是`IPython traitlets`和traitlets是重要的。要处理更改，可以使用小部件的观察方法来注册回调。`observe`的文档字符串可以在下面看到。

In [4]:
print(widgets.Widget.observe.__doc__)

Setup a handler to be called when a trait changes.

        This is used to setup dynamic notifications of trait changes.

        Parameters
        ----------
        handler : callable
            A callable that is called when a trait changes. Its
            signature should be ``handler(change)``, where ``change`` is a
            dictionary. The change dictionary at least holds a 'type' key.
            * ``type``: the type of notification.
            Other keys may be passed depending on the value of 'type'. In the
            case where type is 'change', we also have the following keys:
            * ``owner`` : the HasTraits instance
            * ``old`` : the old value of the modified trait attribute
            * ``new`` : the new value of the modified trait attribute
            * ``name`` : the name of the modified trait attribute.
        names : list, str, All
            If names is All, the handler will apply to all traits.  If a list
            of str, handler wil

### 签名

在文档字符串中提到，注册的回调必须具有`handler(change)`签名，其中`change`是包含有关更改信息的字典。

使用此方法，可以在下面看到如何在更改`IntSlider`值时输出的示例。

In [5]:
int_range = widgets.IntSlider()
display(int_range)

def on_value_change(change):
    print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

1
2


## 链接小部件

通常，您可能只想将小部件属性链接在一起。 属性的同步可以通过简单的方式完成，而不是仅仅使用traitlets事件。

### 链接内核中的traitlets属性

第一种方法是使用traitlets模块中的`link`和`dlink`函数（为方便起见，这两个函数由ipywidgets模块重新导出）。这只适用于我们正在与活动内核交互的情况。

In [6]:
caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
l = widgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption, sliders1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [7]:
caption = widgets.Label(value='Changes in source values are reflected in target1')
source, target1 = widgets.IntSlider(description='Source'),\
                  widgets.IntSlider(description='Target 1')
dl = widgets.dlink((source, 'value'), (target1, 'value'))
display(caption, source, target1)

Label(value='Changes in source values are reflected in target1')

IntSlider(value=0, description='Source')

IntSlider(value=0, description='Target 1')

函数`traitlets.link`和`traitlets.dlink`返回一个`Link`或`DLink`对象。 链接可以通过调用`unlink`方法来中断。

In [8]:
l.unlink()
dl.unlink()

### 将回调注册到内核中的特征更改

由于Python端的窗口小部件属性是`traitlets`，只要模型从前端获取更新时，就可以注册处理更改事件。

传递给观察者的处理程序将被调用一个`change`参数。更改对象至少保存一个类型键和一个名称键，分别对应于通知的类型和触发通知属性的名称。

其他键可能会根据类型的值传递。 在类型改变的情况下，我们也有以下键：

+ `owner`:HasTraits实例
+ `old`:修改过的trait属性的原值
+ `new`:修改过的trait属性的新值
+ `name`:修改后的特征属性的名称

In [9]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=1, description='Slider', max=5, min=-5)

### 链接客户端的小部件属性

在同步traitlets属性时，由于往返服务器端的延迟，您可能会遇到延迟。 您还可以使用链接小部件直接在浏览器中链接小部件属性，无论是单向还是双向。

在没有内核的情况下将小部件嵌入到html网页中时，Javascript链接仍然存在。

In [10]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
range1, range2 = widgets.IntSlider(description='Range 1'),\
                 widgets.IntSlider(description='Range 2')
l = widgets.jslink((range1, 'value'), (range2, 'value'))
display(caption, range1, range2)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=0, description='Range 1')

IntSlider(value=0, description='Range 2')

In [11]:
caption = widgets.Label(value='Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                              widgets.IntSlider(description='Target range 1')
dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))
display(caption, source_range, target_range1)

Label(value='Changes in source_range values are reflected in target_range1')

IntSlider(value=0, description='Source range')

IntSlider(value=0, description='Target range 1')

函数`widgets.jslink`返回一个`Link`小部件。 链接可以通过调用`unlink`方法来中断。

In [12]:
# l.unlink()
# dl.unlink()

### 内核链接和客户端链接的区别

内核链接意味着通过python进行链接。 如果两个滑块在内核中链接，当一个滑块被更改时，浏览器向内核发送一条消息（本例中是python），更新已更改的滑块，然后内核中的链接控件将更改传播到 内核，然后另一个滑块的内核对象向浏览器发送消息以更新浏览器中其他滑块的视图。 如果内核没有运行（如在静态网页中），那么控件将不会被链接。

使用jslink进行链接（即在浏览器端）意味着在Javascript中构建链接。 当更改一个滑块时，在浏览器中运行的Javascript会更改浏览器中其他滑块的值，而无需与内核进行通信。 如果滑块连接到内核对象，则每个滑块将独立更新其内核对象。

要查看两者之间的差异，请转至[ipywidgets文档中此页面的静态版本](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html)，并尝试底部附近的滑块。在`link`和`dlink`内核链接不再链接，但在浏览器中与jslink和jsdlink链接的链接仍然是链接的。

## 持续更新

一些小部件在持续更新值之间提供持续更新属性的选择，或者在用户提交值时（例如，按Enter或远离控件导航）更新值。 在下一个示例中，我们看到“延迟”控件仅在用户完成拖动滑块或提交文本框后才传输其值。 “连续”控件在更改时持续传输它们的值。 尝试在每个文本框中输入一个两位数的数字，或拖动每个滑块，以查看其差异。

In [13]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

滑块，文本和Textarea控件默认为`continuous_update = True`。 用于输入整数或浮点数的IntText和其他文本框默认为`continuous_update = False`（因为通常您需要在按提交值或通过导航框提交值之前键入整个数字）。